In [1]:
from matplotlib import pyplot as plt
import networkx as nx
import numpy as np

# Erdos Renyi

In [ ]:
N = 500
k = 0.8

def gen_erdos(N, k):
    G = nx.erdos_renyi_graph(N, k / N)

    options = {
        "node_color": "red",
        'node_size': 5,
    }
    nx.draw(G, pos=nx.circular_layout(G), alpha=0.5, **options)
    plt.savefig('p1_' + str(int(k)) + 'graph.jpg')
    plt.show()

    g_degrees = np.zeros(N)
    g_clustering = np.zeros(N)
    d_degree = G.degree
    d_clustering = nx.clustering(G)
    for i in range(N):
        g_degrees[i] = d_degree[i]
        g_clustering[i] = d_clustering[i]

    plt.hist(g_degrees, bins=15)
    plt.title('degree distribution for <k> = ' + str(k))
    plt.savefig('p1_'+str(int(k))+'deg.jpg', bbox_inches='tight')
    plt.show()
    plt.hist(g_clustering, bins=15)
    plt.title('clustering distribution for <k> = ' + str(k))
    plt.savefig('p1_'+str(int(k))+'clust.jpg', bbox_inches='tight')
    plt.show()
    print(np.mean(g_degrees))
    print(np.mean(g_clustering))

gen_erdos(500, 0.8)
gen_erdos(500, 1.0)
gen_erdos(500, 8.0)

# Metropolis

In [125]:
# dist target
def target_dist(x):
    sigma = 1
    return np.exp(- x ** 2 / (2 * sigma ** 2))

def gen_gaus(step, size):
    # Initialize at x=0
    x = 0
    count_accept = 0
    n = 100
    counter = 0
    rands = np.zeros(size)
    allin = np.zeros(size * n)
    # Random choice of y with step
    for i in range(n * size):
        y = x + step * np.random.uniform(-1, 1)
        # metropolis decision
        if np.random.uniform(0, 1) < target_dist(y) / target_dist(x):
            x = y
            count_accept += 1
        # save current
        allin[counter] = x
        # increment counter
        counter += 1
    # pick the random sample
        if counter % n == 0:
            rands[counter // n - 1] = x
    # output
    return rands, count_accept / (n * size), allin

In [185]:
size = 1000
steps = [0.5, 1, 1.6, 2.2, 2.9, 3.9, 5.3, 7.9, 16]
data = [None] * 9
acc_rate = [None] * 9
excesses = [None] * 9
for i in range(len(steps)):
    print(i)
    data[i], acc_rate[i], excesses[i] = gen_gaus(steps[i], size)
print(acc_rate)

# dt = plt.hist(data, density=1, bins=40)

0
1
2
3
4
5
6
7
8
[0.90037, 0.80395, 0.69525, 0.60258, 0.50288, 0.40189, 0.30121, 0.2004, 0.09871]


In [187]:
# Find the auto correlation coefficient for the data
def corr_len(excess):
    n = excess.shape[0]
    auto_cor = np.zeros(40)
    for j in range(0, 40):
        sum = 0
        for i in range(n - j):
            sum += excess[i] * excess[i + j]
        sum /= n - j
        auto_cor[j] = (sum - np.mean(excess) ** 2) / np.var(excess)
    return len(auto_cor[auto_cor > np.exp(-1)])
leng = []
for excess in excesses:
    leng.append(corr_len(excess))
leng

[28, 9, 4, 3, 2, 2, 3, 4, 8]

# Monte Carlo ERF(x) from 0 to 2

In [ ]:
from time import time

In [ ]:
def f(x):
    return np.exp(- x ** 2)

# SS MC with stat error
def SSMC(n):
    start = time()
    mean_f = 0
    mean_f2 = 0
    for i in range(n):
        rand = np.random.uniform(0, 2)
        mean_f += f(rand)
        mean_f2 += f(rand) ** 2

    mean_f2 /= n
    mean_f /= n
    elapsed = time() - start
    err = np.sqrt(mean_f2 - mean_f ** 2 / n)
    result = mean_f * 2

    print("I = %.6f" % result)
    print("stat delta I = %.6f" % err)
    print("real delta I = %.6f" % (result - 0.8820814))
    print("runtime = %.5f" % elapsed)

SSMC(64000)

In [ ]:
# IS MC
def ISMC(n):
    def f(x):
        return np.exp(- x ** 2)

    def g(x):
        return np.exp(-x)
    start = time()
    # The integral for the result
    INT_G = 1 - np.exp(-2)
    range2 = np.exp(-2)
    # random generator with g(x) prob. dist.
    def g_rand(a, b):
        return - np.log(np.random.uniform(a, b))
    # Initialize 
    mean_fog = 0
    mean_fog2 = 0

    for i in range(n):
        rand = g_rand(range2, 1)
        mean_fog += f(rand) / g(rand)
        mean_fog2 += (f(rand) / g(rand)) ** 2

    mean_fog /= n
    mean_fog2 /= n

    result = INT_G * mean_fog
    err = np.sqrt((mean_fog2 - mean_fog ** 2) / n)
    elapsed = time() - start
    # Report
    print("I = %.6f" % result)
    print("stat delta I = %.6f" % err)
    print("real delta I = %.6f" % (result - 0.8820814))
    print("runtime = %.5f" %elapsed)
    
ISMC(1000)